In [1]:
from PIL import Image

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Create a DNN model
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(34,)))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.load_weights('./MyModel')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               4480      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 12,801
Trainable params: 12,801
Non-trainable params: 0
_________________________________________________________________
None


In [5]:
from tensorflow.keras.models import load_model, Sequential
model = load_model('./MyModel.h5')
print("model loaded successfully")
print(model.summary())

model loaded successfully
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               4480      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 12,801
Trainable params: 12,801
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
import cv2
import numpy as np 
from skimage import color, feature
from scipy.stats import kurtosis
from PIL import Image



In [7]:
import cv2
import numpy as np 
from skimage import color, feature
from scipy.stats import kurtosis
from PIL import Image, ImageDraw
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout



In [8]:
def extract_features(img):
    # Convert the image to different color spaces
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lab_img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

    # Split the channels
    b, g, r ,x = cv2.split(img)
    h, s, v = cv2.split(hsv_img)
    l, a, b_human = cv2.split(lab_img)

    # Calculate mean values
    mean_r, mean_g, mean_b = np.mean(r), np.mean(g), np.mean(b)
    mean_h, mean_s, mean_v = np.mean(h), np.mean(s), np.mean(v)
    mean_l, mean_a, mean_bhuman = np.mean(l), np.mean(a), np.mean(b_human)

    # Calculate standard deviations
    std_r, std_g, std_b = np.std(r), np.std(g), np.std(b)
    std_h, std_s, std_v = np.std(h), np.std(s), np.std(v)
    std_l, std_a, std_bhuman = np.std(l), np.std(a), np.std(b_human)

    # Calculate variances
    var_r, var_g, var_b = np.var(r), np.var(g), np.var(b)
    var_h, var_s, var_v = np.var(h), np.var(s), np.var(v)
    var_l, var_a, var_bhuman = np.var(l), np.var(a), np.var(b_human)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    kurtosis_r, kurtosis_g, kurtosis_b = kurtosis(r), kurtosis(g), kurtosis(b)
    kurtosis_r_single = np.mean(kurtosis_r)
    kurtosis_g_single = np.mean(kurtosis_g)
    kurtosis_b_single = np.mean(kurtosis_b)
    # Other texture features
    glcm = feature.graycomatrix(gray_img, [1], [0], symmetric=True, normed=True)
    energy = feature.graycoprops(glcm, prop='energy')[0, 0]
    ASM = feature.graycoprops(glcm, prop='ASM')[0, 0]
    contrast = feature.graycoprops(glcm, prop='contrast')[0, 0]
    homogeneity = feature.graycoprops(glcm, prop='homogeneity')[0, 0]

    features_dict = {
        "mean_r": mean_r, "mean_g": mean_g, "mean_b": mean_b,
        "mean_h": mean_h, "mean_s": mean_s, "mean_v": mean_v,
        "mean_l": mean_l, "mean_a": mean_a, "mean_bhuman": mean_bhuman,
        "std_r": std_r, "std_g": std_g, "std_b": std_b,
        "std_h": std_h, "std_s": std_s, "std_v": std_v,
        "std_l": std_l, "std_a": std_a, "std_bhuman": std_bhuman,
        "var_r": var_r, "var_g": var_g, "var_b": var_b,
        "var_h": var_h, "var_s": var_s, "var_v": var_v,
        "var_l": var_l, "var_a": var_a, "var_bhuman": var_bhuman,
        "energy": energy, "ASM": ASM, "contrast": contrast, "homogeneity": homogeneity,
        "kurtosis_r_single": kurtosis_r_single, "kurtosis_g_single": kurtosis_g_single, "kurtosis_b_single": kurtosis_b_single
    }
    
    return features_dict


In [9]:
def extract_features_and_color(image_path, output_folder_path, model):
    # Open the large image
    large_image = Image.open(image_path)

    # Get the size of the large image
    width, height = large_image.size

    # Define the size of the smaller images
    tile_size = 16
    arrr = []
    
    # Create a copy of the original image to draw on
    img_draw = large_image.copy()
    draw = ImageDraw.Draw(img_draw)

    # Loop through the large image and create smaller tiles
    for y in range(0, height, tile_size):
        for x in range(0, width, tile_size):
            # Crop a 32x32 region from the large image
            tile = large_image.crop((x, y, x + tile_size, y + tile_size))
            
            # Convert the tile to a numpy array
            tile_np = np.array(tile)
            thickness = 2
            # Extract features using the defined function
            features = extract_features(tile_np)
            
            # Predict using the model and check if p > 0.5
            prediction = model.predict(np.array([list(features.values())]))
            
            # Append the prediction result to arrr
            arrr.append(prediction)
            
            # Color the tile based on the prediction
            color = (255, 0, 0) if prediction > 0.05 else (0, 255, 0)
            for i in range(thickness):
                draw.rectangle([x - i, y - i, x + tile_size + i, y + tile_size + i], outline=color)

    img_draw.save(output_folder_path)
    return arrr

In [31]:
# Example usage
input_image_path = './sample/xyu.png'
output_folder_path = "./sample/finalO.png"
# Replace 'your_model' with your actual model
your_model = None
ppee =  extract_features_and_color(input_image_path, output_folder_path, model=model)

1/1 [==============================] - 0s 18ms/step


In [32]:
from PIL import Image, ImageDraw
import numpy as np
from shapely.geometry import LineString, Polygon

def extract_features_and_color_1line(input_image_path, output_folder_path, model, line_points):
    # Open the large image
    large_image = Image.open(input_image_path)
    # Get the size of the large image
    width, height = large_image.size
    # Define the size of the smaller images
    tile_size = 16
    arrr = []
    # Create a copy of the original image to draw on
    img_draw = large_image.copy()
    draw = ImageDraw.Draw(img_draw)
    # Loop through the large image and create smaller tiles
    for y in range(0, height, tile_size):
        for x in range(0, width, tile_size):
            # Crop a 16x16 region from the large image
            tile = large_image.crop((x, y, x + tile_size, y + tile_size))

            # Convert the tile to a numpy array
            tile_np = np.array(tile)
            thickness = 2

            # Create a LineString object from the input points
            line = LineString(line_points)

            # Convert the tile to a Shapely Polygon
            tile_polygon = Polygon([(x, y), (x + tile_size, y), (x + tile_size, y + tile_size), (x, y + tile_size)])

            # Check for intersection between the line and the tile
            if line.intersects(tile_polygon):
                # Append the prediction result to arrr
                features = extract_features(tile_np)

                # Predict using the model and check if p > 0.5
                prediction = model.predict(np.array([list(features.values())]))
                arrr.append(prediction)

                # Color the tile based on the prediction
                color = (255, 0, 0) if prediction > 0.5 else (0, 255, 0)
                for i in range(thickness):
                    draw.rectangle([x - i, y - i, x + tile_size + i, y + tile_size + i], outline=color)

            else:
                arrr.append(0)  # If no intersection, append 0 to arrr
    draw.line(line_points, fill=(0, 0, 255), width=4)
    img_draw.save(output_folder_path)
    
    return arrr


In [45]:
output_folder_path = './sample/final1.png'
pp=extract_features_and_color_1line(input_image_path, output_folder_path, model=model,line_points=[(160,0),(620,350)])

1/1 [==============================] - 0s 15ms/step


In [47]:
from PIL import Image, ImageDraw
import numpy as np
from shapely.geometry import LineString, Polygon

def extract_features_and_color_NLines(input_image_path, output_folder_path, model, lines):
    # Open the large image
    large_image = Image.open(input_image_path)
    # Get the size of the large image
    width, height = large_image.size
    # Define the size of the smaller images
    tile_size = 16
    arrr = []
    # Create a copy of the original image to draw on
    img_draw = large_image.copy()
    draw = ImageDraw.Draw(img_draw)

    # Draw blue lines on the image for each line in the list
    for line_points in lines:
        draw.line(line_points, fill=(0, 0, 255), width=4)

    # Loop through the large image and create smaller tiles
    for y in range(0, height, tile_size):
        for x in range(0, width, tile_size):
            # Crop a 16x16 region from the large image
            tile = large_image.crop((x, y, x + tile_size, y + tile_size))

            # Convert the tile to a numpy array
            tile_np = np.array(tile)
            thickness = 2

            # Convert the tile to a Shapely Polygon
            tile_polygon = Polygon([(x, y), (x + tile_size, y), (x + tile_size, y + tile_size), (x, y + tile_size)])

            # Check for intersection between each line and the tile
            for line_points in lines:
                line = LineString(line_points)
                if line.intersects(tile_polygon):
                    # Append the prediction result to arrr
                    features = extract_features(tile_np)

                    # Predict using the model and check if p > 0.5
                    prediction = model.predict(np.array([list(features.values())]))
                    arrr.append(prediction)

                    # Color the tile based on the prediction
                    color = (255, 0, 0) if prediction > 0.5 else (0, 255, 0)
                    for i in range(thickness):
                        draw.rectangle([x - i, y - i, x + tile_size + i, y + tile_size + i], outline=color)
                    break  # Break out of the loop once an intersection is found

            else:
                arrr.append(0)  # If no intersection, append 0 to arrr

    img_draw.save(output_folder_path)
    return arrr


In [59]:
output_folder_path = './sample/final2.png'
xx = extract_features_and_color_NLines(input_image_path, output_folder_path, model=model,lines=[[(160,0),(720,400)],[(720,400),(800,360)]])

1/1 [==============================] - 0s 17ms/step
